# Super-Resolution Sea Surface Salinity (SSS) Data Description

This notebook provides an overview and tools for working with high-resolution sea surface salinity (SSS) data from a 1/48-degree MITgcm simulation. The data is stored in a compact "shrunk" format and can be restored to full 2D fields using a mask and utility functions.

## Objectives
- Load and process global SSS data.
- Visualize salinity fields for the eastern and western hemispheres.
- Document the dataset’s history and structure.

## Prerequisites
- Python packages: `earthaccess`, `matplotlib`, `xarray`, `requests`, `numpy`, `scipy`
- Access to the data directory: `/home/jovyan/shared-public/swot_shared/super_resolution/SSS/`

## History

- **06/29/2024**:
  - Uploaded SSS data description to `/home/jovyan/shared-public/swot_shared/super_resolution/SSS/`.
  - Added `utils.py` with the `mds2d` function for loading and mapping data.

In [ ]:
import earthaccess
import matplotlib.pyplot as plt
import xarray as xr
import requests
import numpy as np
import utils
from scipy import signal
from pathlib import Path

# Define base data path
DATA_PATH = Path('/home/jovyan/shared-public/swot_shared/super_resolution/SSS/')

# Check if data path exists
if not DATA_PATH.exists():
    raise FileNotFoundError(f'Data directory not found at {DATA_PATH}. Please verify the path.')

## Global SSS Data - Added 06/29/2024

### Description
- **Location**: `/home/jovyan/shared-public/swot_shared/super_resolution/SSS/`
- **Resolution**: 1/48-degree (~2 km), full grid 12960 x 8640
- **Format**: Binary (big-endian float32, `>f4`)
- **Details**: 
  - Data is "shrunk" by removing zero values over continents to save space.
  - Use the mask file `hFacC_k0.data` (`hfacc != 0` for wet cells) to restore the full grid.
  - The `mds2d` function in `utils.py` splits the data into eastern and western hemispheres.

### Files
- **Mask**: `hFacC_k0.data` - Grid mask for wet/dry cells.
- **Sample Data**: `SSS.0001400112.shrunk` - Example SSS snapshot.

In [ ]:
# Load mask and SSS data
try:
    mask = np.fromfile(DATA_PATH / 'hFacC_k0.data', '>f4')
    sss_shrunk = np.fromfile(DATA_PATH / 'SSS.0001400112.shrunk', '>f4')
except FileNotFoundError as e:
    print(f'Error: {e}')
    raise

# Restore full grid
if len(sss_shrunk) != np.sum(mask != 0):
    raise ValueError('Mismatch between shrunk data and mask wet cells.')
mask[mask != 0] = sss_shrunk
mask[mask == 0] = np.nan

# Split into 2D fields
sss_east, sss_west = utils.mds2d(mask)
print(f'Eastern Hemisphere Shape: {sss_east.shape}')
print(f'Western Hemisphere Shape: {sss_west.shape}')

In [ ]:
# Visualize the data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), dpi=100)

im1 = ax1.imshow(sss_east, cmap='viridis', interpolation='nearest')
ax1.set_title('Eastern Hemisphere SSS')
ax1.set_xlabel('Longitude Index')
ax1.set_ylabel('Latitude Index')
plt.colorbar(im1, ax=ax1, label='Salinity (psu)')

im2 = ax2.imshow(sss_west, cmap='viridis', interpolation='nearest')
ax2.set_title('Western Hemisphere SSS')
ax2.set_xlabel('Longitude Index')
ax2.set_ylabel('Latitude Index')
plt.colorbar(im2, ax=ax2, label='Salinity (psu)')

plt.tight_layout()
plt.show()

## Spectral Analysis (Optional)

This section demonstrates a basic spectral analysis of the SSS data, comparing the raw data to a smoothed version. It requires a `spectrum` function (not provided in the original notebook), so it’s commented out by default.

### Notes
- The smoothing uses a 10x10 grid box average (~20 km).
- Define a `spectrum` function if you wish to use this section.

In [ ]:
# Define a simple spectrum function if not available
def spectrum(data, fs=1):
    f, p = signal.welch(data.flatten(), fs=fs, nperseg=256)
    return f, p

# Subset and smooth data
sst_subset = sss_east[:280, :190]  # Using SSS as a proxy (rename if SST data exists)
sst_smoothed = sst_subset.reshape(28, 10, 19, 10).mean(axis=1).mean(axis=-1)

# Plot smoothed comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax1.imshow(sst_subset, cmap='viridis')
ax1.set_title('Raw SSS Subset')
ax2.imshow(sst_smoothed, cmap='viridis')
ax2.set_title('Smoothed SSS (20 km)')
plt.tight_layout()
plt.show()

# Spectral analysis
plt.figure(figsize=(8, 5))
f_raw, p_raw = spectrum(sst_subset)
plt.loglog(f_raw, p_raw, label='Raw SSS')
f_smooth, p_smooth = spectrum(sst_smoothed, fs=20)
plt.loglog(f_smooth, p_smooth, label='Smoothed SSS')
plt.xlabel('Frequency')
plt.ylabel('Power Spectral Density')
plt.title('Spectral Analysis of SSS')
plt.legend()
plt.grid(True)
plt.show()